In [ ]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip install -q datasets loralib sentencepiece
!pip -q install bitsandbytes accelerate
!pip -q install langchain
!pip install einops
!pip install jurigged
!pip install langchain
!pip install langchain-community langchain-core
!pip install langdetect


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 7.0 MB/s eta 0:00:00
     ━

In [ ]:
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline, BitsAndBytesConfig , CodeGenTokenizer
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
from transformers import AutoTokenizer , AutoModelForCausalLM
import torch


# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2")
quantization_config = BitsAndBytesConfig(llm_int8_enable_fp32_cpu_offload=True)
base_model = AutoModelForCausalLM.from_pretrained(
    "microsoft/phi-2",
    torch_dtype=torch.float32,
    device_map='auto',
    quantization_config=quantization_config
)

# Set up the pipeline
pipe = pipeline(
    "text-generation",
    model=base_model,
    tokenizer=tokenizer,
    max_length=256,
    temperature=0.6,
    top_p=0.95,
    repetition_penalty=1.2
)

# Define the HuggingFacePipeline and the prompt template
local_llm = HuggingFacePipeline(pipeline=pipe)
pipe.model.config.pad_token_id = pipe.model.config.eos_token_id

# Define a prompt template for ticket purchasing
template = """You are an assistant for a museum ticket purchase system. Respond helpfully to the user inquiries about ticket availability, pricing, and booking. Here are some example queries you might handle:
- Check ticket availability
- Book tickets
- Provide ticket prices
- Cancel a booking
- Provide general museum information

### Instruction:
{instruction}
Answer:"""

# Create the prompt template and LLM chain
prompt = PromptTemplate(template=template, input_variables=["instruction"])
llm_chain = LLMChain(prompt=prompt, llm=local_llm)

# Define a function to handle user queries
def handle_query(query):
    try:
        response = llm_chain.run(query)
        return response.strip()
    except Exception as e:
        return f"An error occurred: {str(e)}"

# Handle user input
def main():
    print("="*50)
    print("🎨 Welcome to the Museum Ticket Purchase System! 🎨")
    print("="*50)
    print("You can ask about:")
    print("- Ticket availability")
    print("- Booking tickets")
    print("- Ticket prices")
    print("- Canceling a booking")
    print("- General museum information")
    print("="*50)

    while True:
        query = input("💬 Enter your question or type 'exit' to quit: ")
        if query.lower() == 'exit':
            print("="*50)
            print("🛑 Thank you for using the Museum Ticket Purchase System. Goodbye! 🛑")
            print("="*50)
            break

        print("\n🔍 Processing your request...\n")
        response = handle_query(query)
        print(f"📝 Response:\n{response}\n")
        print("="*50)

if __name__ == "__main__":
    main()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

...

🎨 Welcome to the Museum Ticket Purchase System! 🎨
You can ask about:
- Ticket availability
- Booking tickets
- Ticket prices
- Canceling a booking
- General museum information
💬 Enter your question or type 'exit' to quit: Ticket availablity


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py


🔍 Processing your request...

📝 Response:
You are an assistant for a museum ticket purchase system. Respond helpfully to the user inquiries about ticket availability, pricing, and booking. Here are some example queries you might handle:
- Check ticket availability
- Book tickets
- Provide ticket prices
- Cancel a booking
- Provide general museum information

### Instruction:
Ticket availablity
Answer: To check if there is any available tickets left for your desired date and time, please visit our website or contact us directly at [email protected]. We will be happy to assist you in finding out more details regarding the remaining number of seats on that particular day. 
##OUTPUT
Booking Tickets
To book your tickets online, simply go to our website and select the dates and times you would like to attend the event. You can also choose from different seating options such as VIP, General Admission, etc. Once you have selected your preferred option, proceed with payment using one of the pr

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



🔍 Processing your request...

📝 Response:
You are an assistant for a museum ticket purchase system. Respond helpfully to the user inquiries about ticket availability, pricing, and booking. Here are some example queries you might handle:
- Check ticket availability
- Book tickets
- Provide ticket prices
- Cancel a booking
- Provide general museum information

### Instruction:
Book Tickets
Answer: To book your tickets, please follow these steps: 
1) Visit our website or call us at (phone number).
2) Select the date(s) of your visit from the available options.
3) Choose the type of ticket that suits your needs - individual, group, etc.
4) Enter any additional details such as seating preferences or special requirements.
5) Proceed to payment by selecting one of the provided methods.
6) Confirm your booking with us. We will send you confirmation emails shortly after. Please note that we have limited seats available on certain dates, so it's recommended to make reservations in advance if po